In [ ]:
ARs = ARs.reset_index()
idx_Fem_sales = ARs[(ARs["Gender"]=="Female") & (ARs["Trade Type"]== "S - Sale")].index
idx_Fem_salesOE = ARs[(ARs["Gender"]=="Female") & (ARs["Trade Type"]== "S - Sale+OE")].index
idx_Fem_purchases = ARs[(ARs["Gender"]=="Female") & (ARs["Trade Type"].str.contains("P - Purchase"))].index
idx_Male_sales = ARs[(ARs["Gender"]=="Male") & (ARs["Trade Type"]== "S - Sale")].index
idx_Male_purchases = ARs[(ARs["Gender"]=="Male") & (ARs["Trade Type"].str.contains("P - Purchase"))].index
idx_Male_salesOE = ARs[(ARs["Gender"]=="Male") & (ARs["Trade Type"]=="S - Sale+OE")].index

idx_lists = [idx_Fem_sales,idx_Fem_salesOE,idx_Fem_purchases,idx_Male_sales,idx_Male_purchases,idx_Male_salesOE]

In [ ]:
#Dont think we need this anymore


group_names = {0:"Female, Sales",1:"Female, Sales + OE",2:"Female, Purchases",3:"Male, Sales",4:"Male, Purchases",5:"Males, Sales + OE"}

if RUN_DESCR_STATS:
    stats = pd.DataFrame()
    
    i = 0
    
    for idx in idx_lists:
        print(len(idx))
        temp = pd.DataFrame({"Day":[item for item in range(-20,21)]})
        group = CARs.iloc[idx]
        mean = group.mean(axis=0).to_frame(name="mean").reset_index() 
        median = group.median(axis=0).to_frame(name="median").reset_index() 
        std = group.std(axis=0).to_frame(name="std").reset_index()
        mini = (group.min(axis=0).to_frame(name="min").reset_index())
        maxi= (group.max(axis=0).to_frame(name="max").reset_index())

        temp = temp.merge(mean, left_on="Day",right_on=mean["index"].astype("int64"))
        temp=temp.merge(median,left_on="Day",right_on=median["index"].astype("int64"))
        temp=temp.merge(std,left_on="Day",right_on=std["index"].astype("int64"))
        temp=temp.merge(mini,left_on="Day",right_on=mini["index"].astype("int64"))
        temp=temp.merge(maxi,left_on="Day",right_on=maxi["index"].astype("int64"))
        
        
        temp["Group"] = group_names[i]
        i+=1
        temp = temp.loc[:,~temp.columns.str.startswith('index')]
        stats = pd.concat([stats,temp])
        
        stats.to_excel(path+"stats.xlsx")
        stats.to_pickle(path+"stats.pkl")

else: 
    stats = pd.read_pickle(path+"stats.pkl")



In [ ]:
sns.set()
sns.set(style="dark")
sns.set(rc={'figure.figsize':(12,7)})

descr_stats = ["mean","median"]

for d in descr_stats:
    subset = stats[[d,"Day","Group"]]
    for i in group_names:
        data = subset[subset["Group"]==group_names[i]]
        sns.lineplot(data=data,x="Day",y=d,palette="Set2",err_style="bars",label=group_names[i]).set(title=str(d)+' CARs')
    plt.axhline(0,color="black",linewidth=0.5)
    plt.savefig(path+"/Vis/"+d, dpi=600, bbox_inches='tight')
    plt.show()